In [39]:
import json
import pandas as pd
from datetime import datetime, timedelta

today = datetime.now()
weekday = today.weekday()  # Monday is 0 and Sunday is 6

# Calculate the start date (Tuesday)
if weekday >= 1:  # If today is Tuesday or after
    start_date = today - timedelta(days=(weekday - 1))
else:  # If today is before Tuesday
    start_date = today - timedelta(days=(weekday + 6))

# Calculate the end date (Monday)
if weekday <= 0:  # If today is Monday
    end_date = today
else:  # If today is after Monday
    end_date = today + timedelta(days=(7 - weekday))

# Convert datetime to date
start_date = start_date.date()
end_date = end_date.date()

def load_historical_data(start_date, end_date):
    plot_data = []
    with open('data_log.jsonl', 'r') as file:
        for line in file:
            entry = json.loads(line)
            datetime = entry['datetime']
            data = entry['data']
            for index, game_id in data['game_id'].items():
                home_team = data['home'][index]
                away_team = data['away'][index]
                home_win = data['Home Win'][index]
                away_win = data['Away Win'][index]

                # Determine which team has the lower win odds
                if home_win < away_win:
                    # Home team has lower odds, so it gets positive points
                    home_points = data['points'][index]
                    away_points = -data['points'][index]
                else:
                    # Away team has lower odds, so it gets positive points
                    home_points = -data['points'][index]
                    away_points = data['points'][index]

                plot_data.append({
                    'DateTime': datetime,
                    'Team': home_team,
                    'Win': home_win,
                    'Type': 'Home Win',
                    'points': home_points
                })
                plot_data.append({
                    'DateTime': datetime,
                    'Team': away_team,
                    'Win': away_win,
                    'Type': 'Away Win',
                    'points': away_points
                })
    df = pd.DataFrame(plot_data)
    df = df[(pd.to_datetime(df['DateTime']).dt.date>=start_date)&(pd.to_datetime(df['DateTime']).dt.date<=end_date)]
    return df

In [40]:
df = load_historical_data(start_date, end_date)

In [41]:
df

,DateTime,Team,Win,Type,points
0,2023-12-05T12:32:09.355434,Tennessee Titans,575,Home Win,-14.0
1,2023-12-05T12:32:09.355434,Miami Dolphins,-900,Away Win,14.0
2,2023-12-05T12:32:09.355434,Seattle Seahawks,425,Home Win,-13.0
3,2023-12-05T12:32:09.355434,San Francisco 49ers,-600,Away Win,13.0
4,2023-12-05T12:32:09.355434,Los Angeles Rams,270,Home Win,-12.0
...,...,...,...,...,...
14995,2023-12-06T11:57:12.965521,Atlanta Falcons,-140,Away Win,2.0
14996,2023-12-06T11:57:12.965521,Buffalo Bills,110,Home Win,-1.0
14997,2023-12-06T11:57:12.965521,Kansas City Chiefs,-130,Away Win,1.0
14998,2023-12-06T11:57:12.965521,Indianapolis Colts,-110,Home Win,-0.0
